In [ ]:
# %pip install langchain
# %pip install pandas
# %pip install python-dotenv
# %pip install langchain_google_genai


#### *Improting genai model method*

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

#### *Prompt Templete*

In [5]:
from langchain.prompts import ChatPromptTemplate

In [7]:
templete_string = """
Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [15]:
prompt_templete = ChatPromptTemplate.from_template(templete_string)
prompt_templete.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='\nTranslate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [16]:
customer_style = """American English \
in a calm and respectful tone
"""

In [17]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [20]:
customer_messages = prompt_templete.format(
    style=customer_style,
    text=customer_email
)

In [21]:
print(type(customer_messages))

<class 'str'>


In [22]:
print(customer_messages)

Human: 
Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [46]:
response=llm.invoke(customer_messages)
output=response.content

In [49]:
type(output)

str

#### *Output Parsers*
Let's start with defining how we would like the LLM output to look like:

In [50]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [51]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [52]:
review_templete = ChatPromptTemplate.from_template(review_template)

In [53]:
review_msg = review_templete.format(text=customer_review)

In [55]:
response=llm.invoke(review_msg)
output=response.content
output

'```json\n{\n  "gift": true,\n  "delivery_days": 2,\n  "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [56]:
type(output)

str

- *In above output of llm type of out put is string we want out put in json type or any other type lanchain provide output parser for that*

In [57]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [62]:
gift_schema = ResponseSchema(name="gift",type="bool", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",type="int", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value",type="list", description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

In [64]:
response_schema = [gift_schema, delivery_days_schema, price_value_schema]

In [65]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [66]:
format_instructions = output_parser.get_format_instructions()

In [67]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": bool  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": int  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": list  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [68]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [69]:
promt_template_2 = ChatPromptTemplate.from_template(review_template_2)

In [70]:
review_msg_2 = promt_template_2.format(text=customer_review, format_instructions=format_instructions)

In [73]:
response=llm.invoke(review_msg_2)
output=response.content
output

'```json\n{\n  "gift": true,\n  "delivery_days": 2,\n  "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [74]:
type(output)

str

In [77]:
parsed_output=output_parser.parse(output)
parsed_output

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [78]:
type(parsed_output)

dict